In [ ]:
import sys
import os
import importlib

# Ajouter le chemin absolu du dossier 'config'
sys.path.append('../config')
# Importer le module de configuration
import config
# Recharger le module de configuration pour prendre en compte les modifications
importlib.reload(config)
# Utiliser la configuration
cfg = config.cfg

import fnmatch
import shutil
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import numpy as np
import glob

###############################################################################
#GENERATION DU DATASET 
###############################################################################

"""Module permettant de générer les images du Dataset FLAIR et SEG  : 
    FLAIR           = /IMAGES
    SEG             = /LABELS
    FLAIR+SEG       = /MASK
"""

def is_slice_completely_black(slice_image):
    """
    Vérifier si image totalement noir, on ne la conserve pas
    """
    return np.all(slice_image == 0)

def resize_with_padding_and_cropping( image, target_size):
    """
    Redimensionnement et retrait des pixels vide
    """
    
    # Calculer les proportions de redimensionnement
    ratio = min(target_size[0] / image.shape[0], target_size[1] / image.shape[1])
    new_size = (int(image.shape[1] * ratio), int(image.shape[0] * ratio))

    # print(f'{image.shape} - {new_size}')
    
    # Redimensionner l'image tout en conservant les proportions
    resized_image = cv2.resize(image, new_size)

    # Créer une image vide de la taille cible
    target_image = np.zeros(target_size, dtype=image.dtype)
    
    # Calculer les coordonnées pour placer l'image redimensionnée au centre de l'image vide
    start_x = (target_size[1] - new_size[1]) // 2
    start_y = (target_size[0] - new_size[0]) // 2
    
    # Placer l'image redimensionnée au centre de l'image vide
    target_image[start_y:start_y + new_size[1], start_x:start_x + new_size[0]] = resized_image
    
    # Normaliser les valeurs des pixels entre 0 et 255
    normalized_image = ((target_image - np.min(target_image)) / (np.max(target_image) - np.min(target_image)) * 255).astype(np.uint8)
    
    # Remplacer les valeurs NaN par des zéros
    nan_mask = np.isnan(normalized_image)
    normalized_image[nan_mask] = 0
    
    return normalized_image

def delete_all_subdirectories(directory):
    # Parcourir tous les éléments du répertoire
    for item in os.listdir(directory):
        # Créer le chemin absolu de l'élément
        item_path = os.path.join(directory, item)
        # Vérifier si l'élément est un répertoire
        if os.path.isdir(item_path):
            # Supprimer récursivement le répertoire
            shutil.rmtree(item_path)

def purge_and_reinit_rep() :  
    """Nettoyer les répertoires TRAIN, VALID, TESTS """
    # Vérifier si les répertoires de sortie existent, sinon les créer
    
    print("Nettoyage des répertoires en cours....")
    
    # TRAIN------------------------------
    if not os.path.exists(cfg['TRAIN_DIR']):
        os.makedirs(cfg['TRAIN_DIR'] )
        
    if not os.path.exists(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_NII'] ): 
        os.makedirs(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_NII'] )
          
    if not os.path.exists(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_IMAGE'] ):
        os.makedirs(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_IMAGE'] )
        
    if not os.path.exists(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_LABELS'] ):
        os.makedirs(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_LABELS'] )
        
    if not os.path.exists(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_MASK'] ):
        os.makedirs(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_MASK'] )

    # VALIDATION------------------------------
    if not os.path.exists(cfg['VALID_DIR'] ):
        os.makedirs(cfg['VALID_DIR'] )
        
    if not os.path.exists(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_NII'] ): 
        os.makedirs(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_NII'] )
          
    if not os.path.exists(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_IMAGE'] ):
        os.makedirs(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_IMAGE'] )
        
    if not os.path.exists(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_LABELS'] ):
        os.makedirs(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_LABELS'] )
        
    if not os.path.exists(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_MASK'] ):
        os.makedirs(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_MASK'] )
        
  
    # TESTS------------------------------
    if not os.path.exists(cfg['TESTS_DIR'] ):
        os.makedirs(cfg['TESTS_DIR'] )
        
    if not os.path.exists(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_NII'] ): 
        os.makedirs(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_NII'] )
          
    if not os.path.exists(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_IMAGE'] ):
        os.makedirs(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_IMAGE'] )
        
    if not os.path.exists(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_LABELS'] ):
        os.makedirs(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_LABELS'] )
        
    if not os.path.exists(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_MASK'] ):
        os.makedirs(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_MASK'] )
        
    # Supprimer tous les fichiers dans les répertoires de sortie après traitement
    delete_all_subdirectories(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_NII'] )
    delete_all_subdirectories(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_NII'] )
    delete_all_subdirectories(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_NII'] )
    
    directories_to_clean = [
        cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_IMAGE'] , 
        cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_LABELS'] ,
        cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_MASK'] ,  
        cfg['VALID_DIR']  + cfg['OUTPUT_DIR_IMAGE'] , 
        cfg['VALID_DIR']  + cfg['OUTPUT_DIR_LABELS'] ,
        cfg['VALID_DIR']  + cfg['OUTPUT_DIR_MASK'] ,
        cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_IMAGE'] , 
        cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_LABELS'] ,
        cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_MASK'] 
        ]

    for directory in directories_to_clean:
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)

    print("Nettoyage des répertoires terminés")

def split_train_valid_test_NII(pourcentTrain, pourcentValid, pourcentTest, nbNIIGen) :
    """
    Ventiler les fichiers source .NII dans les répertoires TRAIN/VALID/TEST
    """

    print(f"Ventilation des images entre TRAIN/VALID/TEST en cours...")
    
    #Connaitre le nombre de fichier _flair.nii pour la répartition
    count = 0
    
    if nbNIIGen == None :
        for subdir in cfg['DATASET_PATH_SUBDIRS'] :
            subdir_path = os.path.join(cfg['DATASET_PATH'], subdir)
            if os.path.exists(subdir_path) :
                for dirname, _, filenames in os.walk(subdir_path):
                    for filename in filenames:
                        if fnmatch.fnmatch(filename, '*_flair.nii'):
                            count += 1
        
                nbTrain = round(count/100 * pourcentTrain)
                nbValid = round(count/100 * pourcentValid)
                nbTests = count-nbTrain-nbValid
    else:
        nbTrain = round(nbNIIGen/100 * pourcentTrain)
        nbValid = round(nbNIIGen/100 * pourcentValid)
        nbTests = nbNIIGen-nbTrain-nbValid
        
    count = 0
    
    exit_all_loops = False
    
    # Parcourir le répertoire source
    for subdir in cfg['DATASET_PATH_SUBDIRS'] :
        if exit_all_loops:
            break  # Sortir de la boucle externe
        
        subdir_path = os.path.join(cfg['DATASET_PATH'], subdir)
        if os.path.exists(subdir_path) :
            
            for dirname, _, filenames in os.walk(subdir_path):
                # Vérifier s'il y a au moins un fichier flair.nii dans le répertoire
                if any(fnmatch.fnmatch(filename, '*_flair.nii') for filename in filenames):
                    # Créer le répertoire correspondant dans la destination
                    if count < nbTrain :
                        destination_subdir = os.path.join(cfg['TRAIN_DIR']  + cfg['OUTPUT_DIR_NII'] , os.path.basename(dirname))
                    elif count < nbTrain+nbValid :
                        destination_subdir = os.path.join(cfg['VALID_DIR']  + cfg['OUTPUT_DIR_NII'] , os.path.basename(dirname))
                    else:
                        destination_subdir = os.path.join(cfg['TESTS_DIR']  + cfg['OUTPUT_DIR_NII'] , os.path.basename(dirname))  
                
                    count+=1
                    os.makedirs(destination_subdir, exist_ok=True)
                    
                  # Copier uniquement les fichiers SEG et FLAIR
                    for filename in filenames:
                        if filename.endswith("_flair.nii") or filename.endswith("_seg.nii"):
                            shutil.copy2(os.path.join(dirname, filename), destination_subdir)
                            
                    if nbNIIGen is not None and count == nbNIIGen:
                        exit_all_loops = True
                        break  # Sortir de la boucle interne

    print(f"Ventilation des images entre TRAIN/VALID/TEST terminé selon répartition {pourcentTrain}%, {pourcentValid}%, {pourcentTest}%")
    
def generate_dataset():
    """
    Génération des images pour l'entrainement (image issue de FLAIR et SEG)
    """

    print(f"Génération des images en cours...")
    
    directories = [cfg['TRAIN_DIR'] , cfg['TESTS_DIR'] , cfg['VALID_DIR'] ]

    for current_dir in directories:
        nii_files_processed = 0  # Variable pour compter le nombre de fichiers NII traités


        # BOUCLER  SUR LES REPERTOIRES
        for dirname, _, filenames in os.walk(current_dir + cfg['OUTPUT_DIR_NII'] ):
            nii_flair = None
            nii_seg = None
            
            for filename in filenames:
                filepath = os.path.join(dirname, filename)
                
                if filename.endswith('_flair.nii')  :
                    nii_flair = nib.load(filepath).get_fdata()
                    
                    #Rotation de l'image 90° pour affichage "vertical"
                    nii_flair = np.rot90(nii_flair,k = -1)
                    
                    # Limiter le nombre de coupes sur l'axe Z à la taille déterminer par VOLUME_SLICES
                    nii_flair = nii_flair[:, :, :cfg['VOLUME_SLICES']]

                elif filename.endswith('_seg.nii') : 
                    nii_seg = nib.load(filepath).get_fdata()
                    nii_seg = np.rot90(nii_seg, k = -1)
                    nii_seg = nii_seg[:, :, :cfg['VOLUME_SLICES']]
           
            if nii_flair is not None and nii_flair.size != 0:
                # Parcourir les 144 tranches, les redimensionner et les sauvegarder dans OUTPUT_DIR_IMAGE
                for i in range(nii_flair.shape[2]):
                    
                    # Vérifier si la tranche est complètement noire, on bypass
                    if is_slice_completely_black(nii_flair[:, :, i]):
                        continue

                    #####################################
                    # IMAGE FLAIR
                    #####################################
                    # Obtenir la tranche actuelle et la redimensionner avec la fonction resize_with_padding_and_cropping
                    slice_image_flair = resize_with_padding_and_cropping(nii_flair[:, :, i], (cfg['IMG_SIZE'] , cfg['IMG_SIZE'] ))
                    slice_image_flair = cv2.cvtColor(slice_image_flair, cv2.COLOR_GRAY2RGB)
                    output_path_flair = os.path.join(current_dir + cfg['OUTPUT_DIR_IMAGE'] , f'{os.path.splitext(filename)[0]}_{i+1:05}.png')
                    cv2.imwrite(output_path_flair, slice_image_flair)
                
                    #####################################
                    # IMAGE SEG
                    #####################################
                    # Obtenir la tranche de l'image segmentée et la redimensionner
                    slice_image_seg = resize_with_padding_and_cropping(nii_seg[:, :, i], (cfg['IMG_SIZE'] , cfg['IMG_SIZE'] ))
                    slice_image_seg_rgb = cv2.cvtColor(slice_image_seg, cv2.COLOR_GRAY2RGB)  # Convertir en RGB
                   
                    # Créer une image rouge pour les pixels segmentés
                    red_slice = np.zeros_like(slice_image_seg_rgb)
                    red_slice[(slice_image_seg_rgb[..., 0] > 0)]  = (0, 0, 255)  # Rouge
                    
                    # # Enregistrer l'image segmentée avec masque rouge
                    output_path_seg = os.path.join(current_dir + cfg['OUTPUT_DIR_LABELS'] , f'{os.path.splitext(filename)[0]}_{i+1:05}.png')
                    cv2.imwrite(output_path_seg, red_slice)
                    
                    #####################################
                    # IMAGE MASK
                    #####################################
                    # Assurez-vous que les tableaux ont le même type de données
                    slice_image_mask = slice_image_flair.astype(red_slice.dtype)
                
                    # Convertir toutes les valeurs de slice_image_mask_rgb qui ne sont pas noires en blanc (255, 255, 255)
                    slice_image_mask[slice_image_mask[..., 0] != 0] = (255, 255, 255)
                
                    # Trouver les coordonnées où red_slice a une valeur de (0, 0, 255)
                    coords = np.where((red_slice[..., 0] == 0) & (red_slice[..., 1] == 0) & (red_slice[..., 2] == 255))
                
                    # Mettre à jour slice_image_mask_rgb à ces coordonnées pour qu'elles aient une valeur de (0, 0, 255)
                    slice_image_mask[coords] = (0, 0, 255)
                
                    # Enregistrer l'image combinée
                    output_path_combined = os.path.join(current_dir  + cfg['OUTPUT_DIR_MASK'] , f'{os.path.splitext(filename)[0]}_{i+1:05}.png')
                    cv2.imwrite(output_path_combined, slice_image_mask)                  
                    
            nii_files_processed += 1  # Incrémente le compteur du nombre de fichiers NII traités
  
    print(f"Génération des images terminée")
  
####################################################################################

purge_and_reinit_rep()

split_train_valid_test_NII(cfg['SPLIT_TRAIN'],
                            cfg['SPLIT_VALID'],
                            cfg['SPLIT_TESTS'],
                            cfg['NBIMGNII_TO_GENERATE'])

generate_dataset()

